# Test the pipeline with the Hungarian Kalman Filter (WKF).

In [1]:
import os
import numpy as np
import torch
import cv2
from torch.utils.data import DataLoader
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm

# Get the path of the project root
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))

# Add the project root to sys.path if it is not already in sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# Import the ImageSequenceDataset and create_dataloader from the pipeline module
from seismogram_extraction.pipeline import evaluate_filter, create_dataloader
from seismogram_extraction.filters.hungarian_kalman_filter import HungarianKalmanFilter

import torch.multiprocessing as mp
mp.set_start_method('spawn')  # or 'forkserver' if available

from collections import defaultdict


In [2]:
selected = r"\data_2_traces" # "data"
N_traces = 2
batch_size = 5

# Overlap: 0

In [3]:
### Parameters
Dts = [0.1, 0.5, 0.9]
qs = [0.01, 0.1, 1]
sigma_zs = [0.01, 0.1, 1]

sources = [r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-00"]
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\resampled\BE_UCC__HHE_2024-01-01T00-06-00_2024-01-14T00-12-00_1000_1_0-1\overlap_0-00"]

# Initialize result structure
results_table = defaultdict(lambda: defaultdict(dict))

if __name__ == "__main__":
    # Grid search loop
    min_RMSE = np.inf
    best_params = {}
    best_result = None

    for Dt in tqdm(Dts, desc='Dt'):
        for sigma_z in sigma_zs:
            for q in qs:
                rmse_list = []
                rmse_std_list = []
                for source in sources:

                    A = np.array([[1, Dt, Dt**2/2, Dt**3/6],
                                [0, 1, Dt, Dt**2/2],
                                [0, 0, 1, Dt],
                                [0, 0, 0, 1]]).astype(np.float64)
                                                        
                    H = np.array([[1, 0, 0, 0]]).astype(np.float64)

                    Dt2 = Dt ** 2
                    Dt3 = Dt ** 3
                    Dt4 = Dt ** 4
                    Dt5 = Dt ** 5
                    Dt6 = Dt ** 6
                    Dt7 = Dt ** 7

                    Q = q * np.array([
                        [Dt7/252, Dt6/72,  Dt5/30, Dt4/24],
                        [Dt6/72,  Dt5/20,  Dt4/8,  Dt3/6 ],
                        [Dt5/30,  Dt4/8,   Dt3/3,  Dt2/2 ],
                        [Dt4/24,  Dt3/6,   Dt2/2,  Dt    ]
                    ])

                    R = np.array([[sigma_z**2]])

                    # Initial state covariance given all_x_0
                    P = np.zeros((4, 4))
                    P[0, 0] = 1
                    P[1, 1] = 10
                    P[2, 2] = 10
                    P[3, 3] = 10

                    image_folder_path = source + r"\signals"
                    GTs_folder_path = source + r"\ground_truth"
                    output_folder_path = None # don't save

                    RMSE_folder, RMSE_std_folder = evaluate_filter(
                        image_folder_path, GTs_folder_path, output_folder_path,
                        HungarianKalmanFilter(A, H, Q, R), P,
                        batch_size=batch_size, step=1, save=False
                    )
                    rmse_list.append(RMSE_folder)
                    rmse_std_list.append(RMSE_std_folder)

                RMSE = np.mean(rmse_list)
                RMSE_std = np.mean(rmse_std_list)
                results_table[Dt][q][sigma_z] = (RMSE, RMSE_std)

                if RMSE < min_RMSE:
                    min_RMSE = RMSE
                    min_RMSE_std = RMSE_std
                    best_params = {'Dt': Dt, 'q' : q, 'sigma_z': sigma_z}

                    print('Best parameters:', best_params)
                    print('Minimum RMSE:', min_RMSE)
                    print('Minimum RMSE std:', min_RMSE_std)
                            
    print('Best parameters:', best_params)
    print('Minimum RMSE:', min_RMSE)
    print('Minimum RMSE std:', min_RMSE_std)

Dt:   0%|          | 0/3 [00:00<?, ?it/s]

Best parameters: {'Dt': 0.1, 'q': 0.01, 'sigma_z': 0.01}
Minimum RMSE: 8.085921176555402
Minimum RMSE std: 13.250429283032178
Best parameters: {'Dt': 0.1, 'q': 0.01, 'sigma_z': 0.1}
Minimum RMSE: 1.0624583982527578
Minimum RMSE std: 0.5280187024903737
Best parameters: {'Dt': 0.1, 'q': 0.1, 'sigma_z': 0.1}
Minimum RMSE: 0.5218077651971708
Minimum RMSE std: 0.09450861772484064


Dt: 100%|██████████| 3/3 [08:50<00:00, 176.81s/it]


Best parameters: {'Dt': 0.1, 'q': 0.1, 'sigma_z': 0.1}
Minimum RMSE: 0.5218077651971708
Minimum RMSE std: 0.09450861772484064


In [4]:
output_dir = os.path.join("output_2_traces_order4", "HKF", "overlap_0")
os.makedirs(output_dir, exist_ok=True)  

for Dt in Dts:
    latex = []
    latex.append(r"\begin{table}[H]")
    latex.append(r"\centering")
    latex.append(r"\begin{tabular}{c|" + "c" * len(sigma_zs) + "}")
    latex.append(r"$q \backslash \sigma_z$ & " + " & ".join([f"{sigma:.2f}" for sigma in sigma_zs]) + r"\\")
    latex.append(r"\hline")

    for q in qs:
        row = [f"{q:.2f}"]
        for sigma_z in sigma_zs:
            rmse_tuple = results_table[Dt][q].get(sigma_z, None)
            if rmse_tuple is not None:
                rmse, std = rmse_tuple
                row.append(f"{rmse:.3f} | {std:.3f}")
            else:
                row.append("-")
        latex.append(" & ".join(row) + r"\\")

    latex.append(r"\end{tabular}")
    latex.append(rf"\caption{{RMSE for Dt = {Dt:.2f}}}")
    latex.append(rf"\label{{tab:rmse_dt_{str(Dt).replace('.', '_')}}}")
    latex.append(r"\end{table}")
    latex.append("")

    # Write to file
    file_path = os.path.join(output_dir, f"RMSE_table_dt_{str(Dt).replace('.', '_')}.tex")
    with open(file_path, "w") as f:
        f.write("\n".join(latex))

print("LaTeX tables written.")


LaTeX tables written.


In [5]:
sources = [r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-00",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\resampled\BE_UCC__HHE_2024-01-01T00-06-00_2024-01-14T00-12-00_1000_1_0-1\overlap_0-00"]

if __name__ == "__main__": 
    Dt = best_params['Dt']
    q = best_params['q']
    sigma_z = best_params['sigma_z']

    rmse_list = []
    rmse_std_list = []
    for source in sources:

        A = np.array([[1, Dt, Dt**2/2, Dt**3/6],
                                [0, 1, Dt, Dt**2/2],
                                [0, 0, 1, Dt],
                                [0, 0, 0, 1]]).astype(np.float64)
                                                        
        H = np.array([[1, 0, 0, 0]]).astype(np.float64)

        Dt2 = Dt ** 2
        Dt3 = Dt ** 3
        Dt4 = Dt ** 4
        Dt5 = Dt ** 5
        Dt6 = Dt ** 6
        Dt7 = Dt ** 7

        Q = q * np.array([
                        [Dt7/252, Dt6/72,  Dt5/30, Dt4/24],
                        [Dt6/72,  Dt5/20,  Dt4/8,  Dt3/6 ],
                        [Dt5/30,  Dt4/8,   Dt3/3,  Dt2/2 ],
                        [Dt4/24,  Dt3/6,   Dt2/2,  Dt    ]
                    ])

        R = np.array([[sigma_z**2]])

        # Initial state covariance given all_x_0
        P = np.zeros((4, 4))
        P[0, 0] = 1
        P[1, 1] = 10
        P[2, 2] = 10
        P[3, 3] = 10

        image_folder_path = os.path.join(source, "signals")
        GTs_folder_path = os.path.join(source, "ground_truth")

        output_folder_path = os.path.join(source, "processed_HKF_dt{:.2f}_q{:.2f}_sigma_z{:.2f}_order4".format(Dt, q, sigma_z))
        os.makedirs(output_folder_path, exist_ok=True)

        # Evaluate on this source (returns RMSE for the image set)
        RMSE_new, _ = evaluate_filter(
            image_folder_path,
            GTs_folder_path,
            output_folder_path,
            HungarianKalmanFilter(A, H, Q, R), P,
            batch_size=batch_size,
            step=1
        )
        rmse_list.append(RMSE_folder)
        rmse_std_list.append(RMSE_std_folder)

    RMSE = np.mean(rmse_list)
    RMSE_std = np.mean(rmse_std_list)

    print('Final evaluation using best parameters:')
    print('RMSE:', RMSE)
    print('RMSE std:', RMSE_std)

Final evaluation using best parameters:
RMSE: 1067128.4200592448
RMSE std: 812570.0650374788


# Medium Overlap

In [6]:
### Parameters
Dts = [0.1, 0.5, 0.9]
qs = [0.01, 0.1, 1]
sigma_zs = [0.01, 0.1, 1]

sources = [r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-28"]
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\resampled\BE_UCC__HHE_2024-01-01T00-06-00_2024-01-14T00-12-00_1000_1_0-1\overlap_0-38"]

# Initialize result structure
results_table = defaultdict(lambda: defaultdict(dict))

if __name__ == "__main__":
    # Grid search loop
    min_RMSE = np.inf
    best_params = {}
    best_result = None

    for Dt in tqdm(Dts, desc='Dt'):
        for sigma_z in sigma_zs:
            for q in qs:
                rmse_list = []
                rmse_std_list = []
                for source in sources:

                    A = np.array([[1, Dt, Dt**2/2, Dt**3/6],
                                [0, 1, Dt, Dt**2/2],
                                [0, 0, 1, Dt],
                                [0, 0, 0, 1]]).astype(np.float64)
                                                        
                    H = np.array([[1, 0, 0, 0]]).astype(np.float64)

                    Dt2 = Dt ** 2
                    Dt3 = Dt ** 3
                    Dt4 = Dt ** 4
                    Dt5 = Dt ** 5
                    Dt6 = Dt ** 6
                    Dt7 = Dt ** 7

                    Q = q * np.array([
                        [Dt7/252, Dt6/72,  Dt5/30, Dt4/24],
                        [Dt6/72,  Dt5/20,  Dt4/8,  Dt3/6 ],
                        [Dt5/30,  Dt4/8,   Dt3/3,  Dt2/2 ],
                        [Dt4/24,  Dt3/6,   Dt2/2,  Dt    ]
                    ])

                    R = np.array([[sigma_z**2]])

                    # Initial state covariance given all_x_0
                    P = np.zeros((4, 4))
                    P[0, 0] = 1
                    P[1, 1] = 10
                    P[2, 2] = 10
                    P[3, 3] = 10

                    image_folder_path = source + r"\signals"
                    GTs_folder_path = source + r"\ground_truth"
                    output_folder_path = None # don't save

                    RMSE_folder, RMSE_std_folder = evaluate_filter(
                        image_folder_path, GTs_folder_path, output_folder_path,
                        HungarianKalmanFilter(A, H, Q, R), P,
                        batch_size=batch_size, step=1, save=False
                    )
                    rmse_list.append(RMSE_folder)
                    rmse_std_list.append(RMSE_std_folder)

                RMSE = np.mean(rmse_list)
                RMSE_std = np.mean(rmse_std_list)
                results_table[Dt][q][sigma_z] = (RMSE, RMSE_std)

                if RMSE < min_RMSE:
                    min_RMSE = RMSE
                    min_RMSE_std = RMSE_std
                    best_params = {'Dt': Dt, 'q' : q, 'sigma_z': sigma_z}

                    print('Best parameters:', best_params)
                    print('Minimum RMSE:', min_RMSE)
                    print('Minimum RMSE std:', min_RMSE_std)
                            
    print('Best parameters:', best_params)
    print('Minimum RMSE:', min_RMSE)
    print('Minimum RMSE std:', min_RMSE_std)

Dt:   0%|          | 0/3 [00:00<?, ?it/s]

Best parameters: {'Dt': 0.1, 'q': 0.01, 'sigma_z': 0.01}
Minimum RMSE: 50.23615212552592
Minimum RMSE std: 20.564317600383962
Best parameters: {'Dt': 0.1, 'q': 0.01, 'sigma_z': 0.1}
Minimum RMSE: 16.952994134588415
Minimum RMSE std: 17.5752661961638


Dt: 100%|██████████| 3/3 [07:55<00:00, 158.49s/it]

Best parameters: {'Dt': 0.1, 'q': 0.01, 'sigma_z': 0.1}
Minimum RMSE: 16.952994134588415
Minimum RMSE std: 17.5752661961638


In [7]:
output_dir = os.path.join("output_2_traces_order4", "HKF", "overlap_28")
os.makedirs(output_dir, exist_ok=True)  

for Dt in Dts:
    latex = []
    latex.append(r"\begin{table}[H]")
    latex.append(r"\centering")
    latex.append(r"\begin{tabular}{c|" + "c" * len(sigma_zs) + "}")
    latex.append(r"$q \backslash \sigma_z$ & " + " & ".join([f"{sigma:.2f}" for sigma in sigma_zs]) + r"\\")
    latex.append(r"\hline")

    for q in qs:
        row = [f"{q:.2f}"]
        for sigma_z in sigma_zs:
            rmse_tuple = results_table[Dt][q].get(sigma_z, None)
            if rmse_tuple is not None:
                rmse, std = rmse_tuple
                row.append(f"{rmse:.3f} | {std:.3f}")
            else:
                row.append("-")
        latex.append(" & ".join(row) + r"\\")

    latex.append(r"\end{tabular}")
    latex.append(rf"\caption{{RMSE for Dt = {Dt:.2f}}}")
    latex.append(rf"\label{{tab:rmse_dt_{str(Dt).replace('.', '_')}}}")
    latex.append(r"\end{table}")
    latex.append("")

    # Write to file
    file_path = os.path.join(output_dir, f"RMSE_table_dt_{str(Dt).replace('.', '_')}.tex")
    with open(file_path, "w") as f:
        f.write("\n".join(latex))

print("LaTeX tables written.")

LaTeX tables written.


In [8]:
sources = [r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-28",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" +selected + r"\resampled\BE_UCC__HHE_2024-01-01T00-06-00_2024-01-14T00-12-00_1000_1_0-1\overlap_0-28"]
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-06",
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-11",
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-17",
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-22",
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-28",
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-33",
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-39",
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-44",
        #    r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\sines_long\overlap_0-50"]

if __name__ == "__main__": 
    Dt = best_params['Dt']
    q = best_params['q']
    sigma_z = best_params['sigma_z']

    rmse_list = []
    rmse_std_list = []
    for source in sources:

        A = np.array([[1, Dt, Dt**2/2, Dt**3/6],
                                [0, 1, Dt, Dt**2/2],
                                [0, 0, 1, Dt],
                                [0, 0, 0, 1]]).astype(np.float64)
                                                        
        H = np.array([[1, 0, 0, 0]]).astype(np.float64)

        Dt2 = Dt ** 2
        Dt3 = Dt ** 3
        Dt4 = Dt ** 4
        Dt5 = Dt ** 5
        Dt6 = Dt ** 6
        Dt7 = Dt ** 7

        Q = q * np.array([
                        [Dt7/252, Dt6/72,  Dt5/30, Dt4/24],
                        [Dt6/72,  Dt5/20,  Dt4/8,  Dt3/6 ],
                        [Dt5/30,  Dt4/8,   Dt3/3,  Dt2/2 ],
                        [Dt4/24,  Dt3/6,   Dt2/2,  Dt    ]
                    ])

        R = np.array([[sigma_z**2]])

        # Initial state covariance given all_x_0
        P = np.zeros((4, 4))
        P[0, 0] = 1
        P[1, 1] = 10
        P[2, 2] = 10
        P[3, 3] = 10

        image_folder_path = os.path.join(source, "signals")
        GTs_folder_path = os.path.join(source, "ground_truth")

        output_folder_path = os.path.join(source, "processed_HKF_dt{:.2f}_q{:.2f}_sigma_z{:.2f}_order4".format(Dt, q, sigma_z))
        os.makedirs(output_folder_path, exist_ok=True)

        # Evaluate on this source (returns RMSE for the image set)
        RMSE_new, _ = evaluate_filter(
            image_folder_path,
            GTs_folder_path,
            output_folder_path,
            HungarianKalmanFilter(A, H, Q, R), P,
            batch_size=batch_size,
            step=1
        )
        rmse_list.append(RMSE_folder)
        rmse_std_list.append(RMSE_std_folder)

    RMSE = np.mean(rmse_list)
    RMSE_std = np.mean(rmse_std_list)

    print('Final evaluation using best parameters:')
    print('RMSE:', RMSE)
    print('RMSE std:', RMSE_std)

Final evaluation using best parameters:
RMSE: 31094917.80804141
RMSE std: 24893359.234343156


## ALL

In [9]:
### Parameters
Dts = [0.1, 0.5, 0.9]
qs = [0.01, 0.1, 1]
sigma_zs = [0.01, 0.1, 1]

sources = [r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-00",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-06",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-11",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-17",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-22",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-28",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-33"]

# Initialize result structure
results_table = defaultdict(lambda: defaultdict(dict))

if __name__ == "__main__":
    # Grid search loop
    min_RMSE = np.inf
    best_params = {}
    best_result = None

    for Dt in tqdm(Dts, desc='Dt'):
        for sigma_z in sigma_zs:
            for q in qs:
                rmse_list = []
                rmse_std_list = []
                for source in sources:

                    A = np.array([[1, Dt, Dt**2/2, Dt**3/6],
                                [0, 1, Dt, Dt**2/2],
                                [0, 0, 1, Dt],
                                [0, 0, 0, 1]]).astype(np.float64)
                                                        
                    H = np.array([[1, 0, 0, 0]]).astype(np.float64)

                    Dt2 = Dt ** 2
                    Dt3 = Dt ** 3
                    Dt4 = Dt ** 4
                    Dt5 = Dt ** 5
                    Dt6 = Dt ** 6
                    Dt7 = Dt ** 7

                    Q = q * np.array([
                        [Dt7/252, Dt6/72,  Dt5/30, Dt4/24],
                        [Dt6/72,  Dt5/20,  Dt4/8,  Dt3/6 ],
                        [Dt5/30,  Dt4/8,   Dt3/3,  Dt2/2 ],
                        [Dt4/24,  Dt3/6,   Dt2/2,  Dt    ]
                    ])

                    R = np.array([[sigma_z**2]])

                    # Initial state covariance given all_x_0
                    P = np.zeros((4, 4))
                    P[0, 0] = 1
                    P[1, 1] = 10
                    P[2, 2] = 10
                    P[3, 3] = 10

                    image_folder_path = source + r"\signals"
                    GTs_folder_path = source + r"\ground_truth"
                    output_folder_path = None # don't save

                    RMSE_folder, RMSE_std_folder = evaluate_filter(
                        image_folder_path, GTs_folder_path, output_folder_path,
                        HungarianKalmanFilter(A, H, Q, R), P,
                        batch_size=batch_size, step=1, save=False
                    )
                    rmse_list.append(RMSE_folder)
                    rmse_std_list.append(RMSE_std_folder)

                RMSE = np.mean(rmse_list)
                RMSE_std = np.mean(rmse_std_list)
                results_table[Dt][q][sigma_z] = (RMSE, RMSE_std)

                if RMSE < min_RMSE:
                    min_RMSE = RMSE
                    min_RMSE_std = RMSE_std
                    best_params = {'Dt': Dt, 'q' : q, 'sigma_z': sigma_z}

                    print('Best parameters:', best_params)
                    print('Minimum RMSE:', min_RMSE)
                    print('Minimum RMSE std:', min_RMSE_std)
                            
    print('Best parameters:', best_params)
    print('Minimum RMSE:', min_RMSE)
    print('Minimum RMSE std:', min_RMSE_std)

Dt:   0%|          | 0/3 [00:00<?, ?it/s]

Best parameters: {'Dt': 0.1, 'q': 0.01, 'sigma_z': 0.01}
Minimum RMSE: 62977.46373711371
Minimum RMSE std: 85814.13451306579
Best parameters: {'Dt': 0.1, 'q': 0.01, 'sigma_z': 0.1}
Minimum RMSE: 2088.1561061872826
Minimum RMSE std: 2723.4349378700726


Dt:  33%|███▎      | 1/3 [19:00<38:01, 1140.53s/it]

Best parameters: {'Dt': 0.1, 'q': 1, 'sigma_z': 1}
Minimum RMSE: 2087.450669537302
Minimum RMSE std: 3599.8452027040234


Dt:  33%|███▎      | 1/3 [25:35<51:11, 1535.99s/it]


KeyboardInterrupt: 

In [ ]:
output_dir = os.path.join("output_2_traces_order4", "HKF", "overlap_ALL")
os.makedirs(output_dir, exist_ok=True)  

for Dt in Dts:
    latex = []
    latex.append(r"\begin{table}[H]")
    latex.append(r"\centering")
    latex.append(r"\begin{tabular}{c|" + "c" * len(sigma_zs) + "}")
    latex.append(r"$q \backslash \sigma_z$ & " + " & ".join([f"{sigma:.2f}" for sigma in sigma_zs]) + r"\\")
    latex.append(r"\hline")

    for q in qs:
        row = [f"{q:.2f}"]
        for sigma_z in sigma_zs:
            rmse_tuple = results_table[Dt][q].get(sigma_z, None)
            if rmse_tuple is not None:
                rmse, std = rmse_tuple
                row.append(f"{rmse:.3f} | {std:.3f}")
            else:
                row.append("-")
        latex.append(" & ".join(row) + r"\\")

    latex.append(r"\end{tabular}")
    latex.append(rf"\caption{{RMSE for Dt = {Dt:.2f}}}")
    latex.append(rf"\label{{tab:rmse_dt_{str(Dt).replace('.', '_')}}}")
    latex.append(r"\end{table}")
    latex.append("")

    # Write to file
    file_path = os.path.join(output_dir, f"RMSE_table_dt_{str(Dt).replace('.', '_')}.tex")
    with open(file_path, "w") as f:
        f.write("\n".join(latex))

print("LaTeX tables written.")

In [ ]:
sources = [r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-00",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-06",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-11",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-17",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-22",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-28",
           r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction" + selected + r"\sines_long\overlap_0-33"]

if __name__ == "__main__": 
    Dt = best_params['Dt']
    q = best_params['q']
    sigma_z = best_params['sigma_z']

    rmse_list = []
    rmse_std_list = []
    for source in sources:

        A = np.array([[1, Dt, Dt**2/2, Dt**3/6],
                                [0, 1, Dt, Dt**2/2],
                                [0, 0, 1, Dt],
                                [0, 0, 0, 1]]).astype(np.float64)
                                                        
        H = np.array([[1, 0, 0, 0]]).astype(np.float64)

        Dt2 = Dt ** 2
        Dt3 = Dt ** 3
        Dt4 = Dt ** 4
        Dt5 = Dt ** 5
        Dt6 = Dt ** 6
        Dt7 = Dt ** 7

        Q = q * np.array([
                        [Dt7/252, Dt6/72,  Dt5/30, Dt4/24],
                        [Dt6/72,  Dt5/20,  Dt4/8,  Dt3/6 ],
                        [Dt5/30,  Dt4/8,   Dt3/3,  Dt2/2 ],
                        [Dt4/24,  Dt3/6,   Dt2/2,  Dt    ]
                    ])

        R = np.array([[sigma_z**2]])

        # Initial state covariance given all_x_0
        P = np.zeros((4, 4))
        P[0, 0] = 1
        P[1, 1] = 10
        P[2, 2] = 10
        P[3, 3] = 10

        image_folder_path = os.path.join(source, "signals")
        GTs_folder_path = os.path.join(source, "ground_truth")

        output_folder_path = os.path.join(source, "processed_HKF_dt{:.2f}_q{:.2f}_sigma_z{:.2f}_ALL_order4".format(Dt, q, sigma_z))
        os.makedirs(output_folder_path, exist_ok=True)

        # Evaluate on this source (returns RMSE for the image set)
        RMSE_new, _ = evaluate_filter(
            image_folder_path,
            GTs_folder_path,
            output_folder_path,
            HungarianKalmanFilter(A, H, Q, R), P,
            batch_size=batch_size,
            step=1
        )
        rmse_list.append(RMSE_folder)
        rmse_std_list.append(RMSE_std_folder)

    RMSE = np.mean(rmse_list)
    RMSE_std = np.mean(rmse_std_list)

    print('Final evaluation using best parameters:')
    print('RMSE:', RMSE)
    print('RMSE std:', RMSE_std)

In [ ]:
# # Evaluate with raw data
# image_folder_path = [r"D:\Courses\Uclouvain\thesis\code\thesis_Colin\seismogram_curve_extraction\data\seismograms"]

# if __name__ == "__main__":
#     Dt = best_params['Dt']
#     q = best_params['q']
#     sigma_z = best_params['sigma_z']

#     A = np.array([[1, Dt, Dt**2/2],
#                   [0, 1, Dt], 
#                   [0, 0, 1]]).astype(np.float64)
                                                        
#     H = np.array([[1, 0, 0]]).astype(np.float64)

#     Q = q * np.array([[Dt**5/20, Dt**4/8, Dt**3/6],
#                                     [Dt**4/8, Dt**3/3, Dt**2/2],
#                                     [Dt**3/6, Dt**2/2, Dt]])

#     R = np.array([[sigma_z**2]])

#     # Initial state covariance given all_x_0
#     P = np.zeros((3, 3))
#     P[1, 1] = 10
#     P[2, 2] = 10

#     image_folder_path = os.path.join(source, "signals")

#     output_folder_path = os.path.join(source, "processed_HKF_dt{:.2f}_q{:.2f}_sigma_z{:.2f}".format(Dt, q, sigma_z))
#     os.makedirs(output_folder_path, exist_ok=True)

#     # Evaluate on this source (returns RMSE for the image set)
#     _, _ = evaluate_filter(
#             image_folder_path,
#             GTs_folder_path,
#             output_folder_path,
#             HungarianKalmanFilter(A, H, Q, R),
#             batch_size=25,
#             step=1
#         )